### Métodos Numéricos II (2023.1 - T02)
- Aluno: Dario Filipe da Silva Costa
- Matrícula: 422156
- Email: dariocosta@alu.ufc.br

In [1]:
import numpy as np
from matplotlib import pyplot as plt

## Dos exercicios anteriores:

### Funções de Integração

##### Integral pela ideia da Soma de Riemann.

In [2]:
def IntegralRiemann(f=lambda x: 2*x**2+5*x-3, a = 1, b = 2, e = 1e-6):
    '''Calcula a integral da função no intervalo [a, b] usando a ideia 
    do Somatorio de Riemann
        - f: Função que será integrada
        - a: Inicio do intervalo
        - b: Fim do intervalo
        - e: Erro relativo do passo
        '''
    Erro = np.Infinity
    Aold = np.Infinity
    N=1
    while Erro > e:
      N=N*2
      Dx = (b-a)/N
      A = 0
      for k in range(N):
        xk = (a+Dx/2)+Dx*k
        A = A+f(xk)*Dx
      Erro = abs((A-Aold)/A)
      Aold=A
    return A, N

#### Código padrão para as integrais

Ele faz a subdivisão até alcançar a tolerancia, a função da Area dos subintervalos é passada como parametro.

In [3]:
def Integral(f=lambda x: 2*x**2+5*x-3, a = 1, b = 2, 
             Area = lambda func, inicio, fim: func(((inicio+fim)/2))*(fim-inicio),
             e = 1e-6):
    '''Calcula a integral da função `f` no intervalo [a, b] usando a formula de `Area` passada.
    #### Parametros:
      A função de `Area` default é a de Riemann, considera um retangulo.
        - `f`: Função que será integrada.
        - `a`: Inicio do intervalo.
        - `b`: Fim do intervalo.
        - `Area(f, xi, xf)`: Uma função que calcula a Area do subintervalo [xi, xf]
        da função f.
        - `e`: Erro relativo do passo.
      #### Retorno:
        Area e o numero de Iterações do laço
        - `A` e `cIt`
        '''
    Erro = np.Infinity
    Aold = np.Infinity
    N=1
    cIt = 0
    while Erro > e:
      N=N*2
      Dx = ((b-a)/N)
      A = 0
      for k in range(N):
        xi = a+Dx*k
        xf = xi+Dx
        A = A + Area(f, xi, xf)
      if A>1e-7:
        Erro = abs((A-Aold)/A)
      else: #Caso o proprio valor da Area seja 0, não podemos dividir por ela.
        Erro = abs(A-Aold)
      Aold=A
      cIt +=1
    return A, cIt

### Newton Cotes:

#### Formulas Fechadas

In [4]:
def trapezio(f, xi, xf):
  Dx = xf-xi
  return 0.5*Dx*(f(xi)+f(xf))

def simpsonUmTerco(f, xi, xf):
  Dx = xf-xi
  h = Dx/2
  return ((h/3)*(f(xi)+4*f(xi+h)+f(xf)))

def simpsonTrezOitavos(f, xi, xf):
  Dx = xf-xi
  h = Dx/3
  return ((Dx/8)*(f(xi)+3*f(xi+h)+3*f(xi+2*h)+f(xf)))

def regraBoole(f, xi, xf):
  Dx = xf - xi
  h=Dx/4
  return ((Dx/90)*(7*f(xi)+32*f(xi+h)+12*f(xi+2*h)+32*f(xi+3*h)+7*f(xf)))


In [5]:
#Função e intervalo que usaremos para testar
func = lambda x: (np.sin(2 * x)+4*x**2+3*x)**2
(a, b) = (0, 1)

In [6]:
formulasFechadas = {
    'Trapezio': trapezio,
    'Simpson 1/3': simpsonUmTerco,
    'Simpson 3/8': simpsonTrezOitavos,
    'Boole': regraBoole
}

#Chamando a função Integral com os parametros do Dicionario.
for Ai in formulasFechadas.keys():
  print(f'Regra do {Ai}: {Integral(func, a, b, Area = formulasFechadas[Ai])}')

Regra do Trapezio: (17.87647353777083, 11)
Regra do Simpson 1/3: (17.876470758051976, 4)
Regra do Simpson 3/8: (17.876470527104225, 4)
Regra do Boole: (17.876470347803945, 3)


#### Formulas Abertas

In [7]:
def trapezioA(f, xi, xf):
  Dx = xf-xi
  h = Dx/3
  return 0.5*Dx*(f(xi-h)+f(xf+h))

def milne(f, xi, xf):
  Dx = xf-xi
  h = Dx/4
  return ((Dx/3)*(2*f(xi+h)-1*f(xi+2*h)+2*f(xf-h)))

def NCAbertaGrau3(f, xi, xf):
  Dx = xf-xi
  h = Dx/5
  return ((Dx/24)*(11*f(xi+h)+f(xi+2*h)+f(xi+3*h)+11*f(xf-h)))

def NCAbertaGrau4(f, xi, xf): 
  Dx = xf-xi
  h = Dx/6
  return ((Dx/30)*((33/2)*f(xi+h)-21*f(xi+2*h)+39*f(xi+3*h)-21*f(xi+4*h)+(33/2)*f(xf-h)))

In [8]:
formulasAbertas = {
    'Trapezio Aberta': trapezioA,
    'Milne': milne,
    'Newton-Cotes Aberta Grau 3': NCAbertaGrau3,
    'Newton-Cotes Aberta Grau 4': NCAbertaGrau4
}

#Chamando a função Integral com os parametros do Dicionario.
for Ai in formulasAbertas.keys():
  print(f'Regra do {Ai}: {Integral(func, a, b, Area = formulasAbertas[Ai])}')

Regra do Trapezio Aberta: (17.876473271472452, 12)
Regra do Milne: (17.876469978475694, 4)
Regra do Newton-Cotes Aberta Grau 3: (17.876470089404435, 4)
Regra do Newton-Cotes Aberta Grau 4: (17.87647033082215, 3)


### Gauss Legendre

In [9]:
def GL_2p(f, xi, xf):
  xa = lambda alpha: 0.5*(xi+xf) + 0.5*(xf-xi)*alpha
  alpha = [-((1/3)**0.5), (1/3)**0.5]
  w = [1, 1]
  S = 0
  for k in range(len(alpha)):
    S+=f(xa(alpha[k]))*w[k]

  return 0.5*(xf-xi)*S

def GL_3p(f, xi, xf):
  xa = lambda alpha: 0.5*(xi+xf) + 0.5*(xf-xi)*alpha
  alpha = [-(3/5)**0.5, 0, (3/5)**0.5]
  w = [ 5/9, 8/9, 5/9]
  S = 0
  for k in range(len(alpha)):
    S+=f(xa(alpha[k]))*w[k]

  return 0.5*(xf-xi)*S

def GL_4p(f, xi, xf):
  xa = lambda alpha: 0.5*(xi+xf) + 0.5*(xf-xi)*alpha
  alpha = [-0.8611363115940526, -0.3399810435848563,
            0.3399810435848563,  0.8611363115940526]
  w = [ 0.347854845137454, 0.652145,
        0.652145, 0.347854845137454]
  S = 0
  for k in range(len(alpha)):
    S+=f(xa(alpha[k]))*w[k]

  return 0.5*(xf-xi)*S

In [10]:
formulasGL = {
    'Gauss Legendre com 2 pontos': GL_2p,
    'Gauss Legendre com 3 pontos': GL_3p,
    'Gauss Legendre com 4 pontos': GL_4p}

#Chamando a função Integral com os parametros do Dicionario.
for Ai in formulasGL.keys():
  print(f'Regra {Ai}: {Integral(func, a, b, Area = formulasGL[Ai])}')

Regra Gauss Legendre com 2 pontos: (17.876470065027217, 4)
Regra Gauss Legendre com 3 pontos: (17.87647033680088, 3)
Regra Gauss Legendre com 4 pontos: (17.87646761628633, 2)


### Quadraturas de Guass especiais

#### Situação de aplicação:
- Hermite:
> $$\int_{-\infty}^{\infty} e^{-x^2} f(x) dx$$

- Laguerre:
> $$\int_{0}^{\infty} e^{-x} f(x) dx$$

- Chebyshev:
> $$\int_{-1}^{1} \frac{1}{\sqrt{1-x^2}} f(x) dx$$

In [11]:
def GHermite(f, n):
  idx = n-2
  xk = [[ -(1/(2**0.5)), 1/(2**0.5)],
        [ -np.sqrt(3/2), 0, np.sqrt(3/2)],
        [ -1.650680124, -0.5246476233, 0.5246476233, 1.650680124]]
  w = [[ np.sqrt(np.pi)/2, np.sqrt(np.pi)/2],
       [ np.sqrt(np.pi)/6, (2*np.sqrt(np.pi))/3, np.sqrt(np.pi)/6],
       [ 0.08131283545, 0.80491409, 0.80491409, 0.08131283545]]
  S = 0
  for k in range(len(xk[idx])):
    S+=f(xk[idx][k])*w[idx][k]

  return S

def GLaguerre(f, n):
  idx=n-2
  xk = [[ 2-np.sqrt(2), 2+np.sqrt(2)],
        [ 0.4157745568, 2.2942803603, 6.2899450829],
        [ 0.32254768962, 1.74576110116, 4.53662029692, 9.3950709123]]
  w = [[ (1/4)*(2+np.sqrt(2)),  (1/4)*(2-np.sqrt(2))],
       [ 0.7110930099, 0.2785177336, 0.0103892565],
       [ 0.6031541045, 0.357418692435, 0.038887908515, 0.00053929470556]]
  S = 0
  for k in range(len(xk[idx])):
    S+=f(xk[idx][k])*w[idx][k]

  return S

def GChebyshev(f, n):
  idx=n-2
  xk = [[ -1/np.sqrt(2), 1/np.sqrt(2)],
        [ -np.sqrt(3)/2, 0, np.sqrt(3)/2],
        [ (-1/2)*np.sqrt(2+np.sqrt(2)), (-1/2)*np.sqrt(2-np.sqrt(2)), (1/2)*np.sqrt(2-np.sqrt(2)), (1/2)*np.sqrt(2+np.sqrt(2))]]
  w = [[ np.pi/2, np.pi/2],
       [ np.pi/3, np.pi/3, np.pi/3],
       [ np.pi/4, np.pi/4, np.pi/4, np.pi/4]]
  S = 0
  for k in range(len(xk[idx])):

    S+=f(xk[idx][k])*w[idx][k]

  return S

Teste da Gauss-Hemite com a mesma função da aula # 14
$$\int_{-\infty}^{\infty} e^{-x^2} \left(e^{x^2} \cdot 1/\left(\sqrt{2+2 \cdot tanh(x)}\cdot (cosh(x))^2 \right) \right)dx = 2$$

In [12]:
fx_Test = lambda x: np.exp((x**2)) * (1/np.sqrt(2+2*np.tanh(x))) * (1/((np.cosh(x))**2))

form_G_Esp = {
    'Gauss Hermite': GHermite,
    'Gauss Laguerre': GLaguerre,
    'Gauss Chebyshev': GChebyshev}

tipo = 'Gauss Hermite'
for n in [2, 3, 4]:
  print(f'Regra {tipo} com {n} pontos: {form_G_Esp[tipo](fx_Test, n)}')

Regra Gauss Hermite com 2 pontos: 1.5521779998914107
Regra Gauss Hermite com 3 pontos: 1.724593069765156
Regra Gauss Hermite com 4 pontos: 1.8102396507325338


Teste da Gauss-Laguerre:
$$\int_{0}^{\infty} e^{-x} \left(e^{x} \cdot 1/\left(\sqrt{2+2 \cdot tanh(x)}\cdot (cosh(x))^2 \right) \right)dx = 2 - \sqrt{2} \approx 0.58578643762690$$

In [13]:
fx_Test2 = lambda x: np.exp((x)) * (1/np.sqrt(2+2*np.tanh(x))) * (1/((np.cosh(x))**2))

tipo = 'Gauss Laguerre'
for n in [2, 3, 4]:
  print(f'Regra {tipo} com {n} pontos: {form_G_Esp[tipo](fx_Test2, n)}')

Regra Gauss Laguerre com 2 pontos: 0.6435055650310119
Regra Gauss Laguerre com 3 pontos: 0.6010432766965328
Regra Gauss Laguerre com 4 pontos: 0.5842201269912122


Teste da Gauss-Chebyshev:
$$\int_{-1}^{1} \frac{1}{\sqrt{1-x^2}} \left(\frac{\sqrt{1-x^2}}{\left(\sqrt{2+2 \cdot tanh(x)}\cdot (cosh(x))^2 \right) }\right)dx \approx 1.1865002761670$$

In [14]:
fx_Test3 = lambda x: (np.sqrt(1-x**2))/(np.sqrt(2+2*np.tanh(x)))

tipo = 'Gauss Chebyshev'
for n in [2, 3, 4]:
  print(f'Regra {tipo} com {n} pontos: {form_G_Esp[tipo](fx_Test2, n)}')

Regra Gauss Chebyshev com 2 pontos: 1.6686652389885608
Regra Gauss Chebyshev com 3 pontos: 1.7206595259345367
Regra Gauss Chebyshev com 4 pontos: 1.71550245316281


Não senti muita confiança na Chebyshev, mas pode ter sido poucos pontos para esse teste, afinal, só modifiquei a função dos itens anteriores sem fazer a mudança de variável correta.

# Tarefa 7: Tratamento de singularidades
- "Derrubando" a função com exponencial simples e dupla.

- Definir um $c$ de corte nas pontas, de forma a ser insignificante o que vai ser ignorado.

- Calcular internamente a integral com qualquer técnica anterior que seja válida.

In [15]:
def IntExp(f=lambda x: 2*x**2+5*x-3, a = 1, b = 2, 
              e1 = 1e-6, e2=1e-7):

    xs=lambda s: 0.5*(a+b)+0.5*(b-a)*np.tanh(s)
    dx_ds = lambda s: 0.5*((b-a)/((np.cosh(s))**2))

    fxsDx_Ds = lambda s: f(xs(s))*dx_ds(s)

    Erro = np.Infinity
    Aold = np.Infinity

    cIt = 0
    cExpt = 0
    A=0
    c=1

    while Erro > e1 and cIt<10**5:
      cIt+=1
      c=2*(cIt**(1/2))*(0.998**cExpt)
      

      try: # Não entendo bem do controle de exception do python
        #Usando Gauss - Legendre 4 pontos
        A = Integral(f=fxsDx_Ds, a=-c, b=c, Area=GL_4p, e= e2)[0]

        if A==np.inf:
          raise ValueError

        if A>1e-8:
          Erro = abs((A-Aold)/A)
        else: #Caso o proprio valor da Area seja 0, não podemos dividir por ela.
          Erro = abs(A-Aold)
      
        Aold=A

      except (ValueError): #To DO: Melhorar esse controle do c
        print("Problema de representação detectado") 
        cExpt+=1
      
    return A, cIt, c

In [16]:
def IntExpDupla(f=lambda x: 2*x**2+5*x-3, a = 1, b = 2, 
              e1 = 1e-6, e2=1e-7):

    xs=lambda s: 0.5*(a+b)+0.5*(b-a)*np.tanh(0.5*np.pi*np.sinh(s))
    dx_ds = lambda s: 0.5*(b-a)*((0.5*np.pi*np.cosh(s))/(np.cosh(0.5*np.pi*np.sinh(s))**2))

    fxsDx_Ds = lambda s: f(xs(s))*dx_ds(s)

    Erro = np.Infinity
    Aold = np.Infinity

    cIt = 0
    cExpt = 0
    A=0
    c=1

    while Erro > e1 and cIt<10**5:
      cIt+=1
      c=((cIt/2)**(1/3))*(0.998**cExpt)
      

      try: # Não entendo bem do controle de exception do python
        #Usando Gauss - Legendre 4 pontos
        A = Integral(f=fxsDx_Ds, a=-c, b=c, Area=GL_4p, e= e2)[0]

        if A==np.inf:
          raise ValueError

        if A>1e-8:
          Erro = abs((A-Aold)/A)
        else: #Caso o proprio valor da Area seja 0, não podemos dividir por ela.
          Erro = abs(A-Aold)
      
        Aold=A

      except (ValueError): #To DO: Melhorar esse controle do c
        print("Problema de representação detectado")
        cExpt+=1
      
    return A, cIt, c

### Item (1) :

- $$I=\int_{-1}^{1}\frac{1}{\sqrt[3]{x^2}} dx = 6$$

Singularidade no 0, e como $x$ está ao quadrado dá para concluir que é simétrica, logo fazemos 2 vezes de $[0,~ 1]$.

In [17]:
f1 = lambda x: 1/((x**2)**(1/3))
a=0
b=1
A_s, cIt_s, c_s = IntExp(f=f1, a=a, b=b)
A_d, cIt_d, c_d = IntExpDupla(f=f1, a=a, b=b)
print(f'Exponencial Simples: {2*A_s}, com {cIt_s} iterações e c = {c_s}')
print(f'Exponencial Dupla: {2*A_d}, com {cIt_d} iterações e c = {c_d}')

Exponencial Simples: 5.999931308735208, com 73 iterações e c = 17.08800749063506
Exponencial Dupla: 5.999970511642029, com 63 iterações e c = 3.1581797988281894


### Item (2):

- $$I=\int_{-2}^{0}\frac{1}{\sqrt{4-x^2}} dx = \frac{\pi}{2} \approx 1.57079632679$$

Como a singularidade já está nos limites (no "a"$ = -2$), e não no meio. Podemos ir direto.

In [18]:
f2 = lambda x: 1/(np.sqrt(4-x**2))
a=-2
b=0
A_s, cIt_s, c_s = IntExp(f=f2, a=a, b=b)
A_d, cIt_d, c_d = IntExpDupla(f=f2, a=a, b=b)
print(f'Exponencial Simples: {A_s}, com {cIt_s} iterações e c = {c_s}')
print(f'Exponencial Dupla: {A_d}, com {cIt_d} iterações e c = {c_d}')

Exponencial Simples: 1.5707887198385595, com 37 iterações e c = 12.165525060596439
Exponencial Dupla: 1.570791406282023, com 43 iterações e c = 2.7806488832606178
